In [165]:
import pandas as pd
import numpy as np

In [166]:
df = pd.read_csv('C:/Users/chait/Desktop/DJSCE/IPD_Medledger/weather_data_2009_2024.csv')
df['Date'] = pd.to_datetime(df['Date'])

C:\Users\chait\AppData\Local\Temp\ipykernel_15032\2393376386.py:2: UserWarning: Parsing '13-01-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\chait\AppData\Local\Temp\ipykernel_15032\2393376386.py:2: UserWarning: Parsing '14-01-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\chait\AppData\Local\Temp\ipykernel_15032\2393376386.py:2: UserWarning: Parsing '15-01-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\chait\AppData\Local\Temp\ipykernel_15032\2393376386.py:2: UserWarning: Parsing '16-01-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\chait\AppData\Local\Tem

In [167]:
df

,Date,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index
0,2009-01-01,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000
1,2009-02-01,11.072351,51.93023256,Fair,5.000000,8.046700,302.5,CALM,29.193488,44.72093023,51.93023256,0.430465,CLR,0.000000
2,2009-03-01,9.308642,48.75555556,Cloudy,5.129032,8.254357,124.1935484,ESE,29.208444,47.2,48.75555556,0.199333,OVC,-45.155556
3,2009-04-01,11.485507,52.67391304,Cloudy,4.977273,8.010124,119.7727273,E,29.143913,48.63043478,52.67391304,0.380870,OVC,-32.369565
4,2009-05-01,14.056848,57.30232558,Partly Cloudy,4.852941,7.810032,246.1764706,NW,29.093953,52.20930233,57.30232558,0.451628,SCT,-25.093023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,2024-02-19,21.099537,69.97916667,Haze,10.333333,16.629847,104.1666667,E,29.011250,51.72916667,70.0625,1.000000,FEW,0.854167
5192,2024-02-20,21.099537,69.97916667,Haze,4.479167,7.208502,133.4782609,E,29.015417,58.33333333,69.95833333,0.936875,SCT,0.895833
5193,2024-02-21,19.386574,66.89583333,Haze,6.041667,9.723096,282.5531915,W,29.018333,53.10416667,66.91666667,1.073125,FEW,1.437500
5194,2024-02-22,19.066194,66.31914894,Haze,5.127660,8.252148,281.8918919,NW,29.092766,43.89361702,66.31914894,1.574468,CLR,1.340426


In [168]:
def distribute_cases_deaths_weekly(df):
    result = []
    for _, row in df.iterrows():
        total_cases = row['Cases']
        total_deaths = row['Deaths']
        days_in_week = 7
        average_cases = total_cases/days_in_week
        average_deaths = total_deaths/days_in_week
        daily_cases = np.random.randint(0.7*average_cases, high=1.3*average_cases+1, size=days_in_week)
        daily_cases[-1] += total_cases - daily_cases.sum()
        daily_deaths = np.random.randint(0.7*average_deaths, high=1.3*average_deaths+1, size=days_in_week)
        daily_deaths[-1] += total_deaths - daily_deaths.sum()

        if daily_cases[-1] < 0:
            daily_cases[-1] *= -1
        if daily_deaths[-1] < 0:
            daily_deaths[-1] *= -1
        
        for i, (cases, deaths) in enumerate(zip(daily_cases, daily_deaths)):
            result.append({
                'Entity': row['Entity'],
                'Code': row['Code'],
                'Date': row['Date'] + pd.DateOffset(days=i),
                'Cases': cases,
                'Deaths': deaths,
                'Disease': row['Disease']
            })

    return pd.DataFrame(result)

In [169]:
covid = pd.read_csv('daily-covid-cases-deaths.csv')
covid = covid[covid['Entity'] == 'India']
covid.reset_index(inplace=True)
covid.drop(columns=['index'], inplace=True)
covid['Disease'] = 'Covid-19'
covid.rename(columns={'Daily new confirmed cases of COVID-19': 'Cases', 'Daily new confirmed deaths due to COVID-19': 'Deaths', 'Day': 'Date'}, inplace=True)
covid['Date'] = pd.to_datetime(covid['Date'])
covid = covid[covid['Cases'] != 0]
covid = covid[covid['Deaths'] != 0]
covid = distribute_cases_deaths_weekly(covid)
covid

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,2020-03-15,11,0,Covid-19
1,India,IND,2020-03-16,9,0,Covid-19
2,India,IND,2020-03-17,9,0,Covid-19
3,India,IND,2020-03-18,11,0,Covid-19
4,India,IND,2020-03-19,10,0,Covid-19
...,...,...,...,...,...,...
1409,India,IND,2024-02-06,139,1,Covid-19
1410,India,IND,2024-02-07,112,1,Covid-19
1411,India,IND,2024-02-08,146,0,Covid-19
1412,India,IND,2024-02-09,137,0,Covid-19


In [170]:
daily_covid = pd.merge(covid, df, on='Date', how='inner')
daily_covid['Symptoms'] = '["Fever", "Dry cough", "Fatigue", "Shortness of breath", "Muscle or body aches", "Sore throat", "Loss of taste or smell", "Headache", "Chills", "Congestion or runny nose", "Nausea", "Vomiting", "Diarrhea"]'
daily_covid

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2020-03-15,11,0,Covid-19,18.460648,65.22916667,Haze,4.166667,6.705583,222.2580645,CALM,29.180417,54.0625,65.22916667,0.847083,SCT,1.354167,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
1,India,IND,2020-03-16,9,0,Covid-19,19.988426,67.97916667,Haze,4.708333,7.577309,290,WNW,29.178750,54.52083333,68.08333333,1.217500,FEW,1.125000,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
2,India,IND,2020-03-17,9,0,Covid-19,21.284722,70.3125,Haze,6.270833,10.091903,283.5555556,WNW,29.173750,55.375,70.1875,1.231042,CLR,2.062500,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
3,India,IND,2020-03-18,11,0,Covid-19,22.303241,72.14583333,Haze,5.145833,8.281395,270.3030303,W,29.127500,57.20833333,72.0625,1.039167,CLR,2.062500,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
4,India,IND,2020-03-19,10,0,Covid-19,23.564815,74.41666667,Haze,3.791667,6.102081,219.3939394,CALM,29.031875,59.91666667,74.625,0.994792,FEW,1.916667,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,India,IND,2024-01-29,246,2,Covid-19,14.113475,57.40425532,Fog,4.872340,7.841252,173.9130435,W,29.221915,52,57.40425532,0.636809,CLR,1.042553,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
1072,India,IND,2024-01-30,180,1,Covid-19,12.695035,54.85106383,Fog,3.042553,4.896503,261.9444444,WSW,29.201702,52.4893617,54.85106383,0.302340,OVC,0.638298,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
1073,India,IND,2024-01-31,229,2,Covid-19,12.696759,54.85416667,Fog,3.791667,6.102081,122.1875,CALM,29.168125,53.54166667,54.85416667,0.274167,OVC,0.604167,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."
1074,India,IND,2024-02-01,187,1,Covid-19,11.076389,51.9375,Mist,3.875000,6.236192,247.826087,WNW,29.223750,46.54166667,51.9375,0.797917,CLR,0.833333,"[""Fever"", ""Dry cough"", ""Fatigue"", ""Shortness o..."


In [171]:
daily_covid.to_csv('Daily/daily_covid.csv')
daily_covid['Cases'].sum()

44680498

In [172]:
influenza = pd.read_csv('influenza.csv')
influenza = influenza[influenza['Country'] == 'India']
influenza.reset_index(inplace=True)
influenza.drop(columns=['index'], inplace=True)
influenza_sum = influenza.iloc[:, 3:].sum(axis=1)
influenza = pd.concat([influenza.iloc[:, :2], influenza_sum], axis=1)
influenza['Code'] = 'IND'
influenza['Disease'] = 'Influenza'
influenza['Deaths'] = np.nan
influenza.rename(columns={'Country': 'Entity', 0: 'Cases', 'Month': 'Date'}, inplace=True)
influenza = influenza[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
influenza['Date'] = pd.to_datetime(influenza['Date'])
influenza

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,2009-01-01,486.000000,NaN,Influenza
1,India,IND,2009-02-01,521.921569,NaN,Influenza
2,India,IND,2009-03-01,272.985075,NaN,Influenza
3,India,IND,2009-04-01,0.000000,NaN,Influenza
4,India,IND,2009-05-01,0.000000,NaN,Influenza
...,...,...,...,...,...,...
175,India,IND,2023-08-01,22931.610600,NaN,Influenza
176,India,IND,2023-09-01,11526.537825,NaN,Influenza
177,India,IND,2023-10-01,22434.966998,NaN,Influenza
178,India,IND,2023-11-01,5459.485526,NaN,Influenza


In [173]:
monthly_influenza = pd.merge(influenza, df, on='Date', how='inner')
monthly_influenza['Symptoms'] = '["Fever", "Cough", "Sore throat", "Runny or stuffy nose", "Muscle or body aches", "Headache", "Fatigue", "Vomiting and diarrhea (more common in children)"]'
monthly_influenza.to_csv('Daily/monthly_influenza.csv')
monthly_influenza

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2009-01-01,486.000000,NaN,Influenza,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
1,India,IND,2009-02-01,521.921569,NaN,Influenza,11.072351,51.93023256,Fair,5.000000,8.046700,302.5,CALM,29.193488,44.72093023,51.93023256,0.430465,CLR,0.000000,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
2,India,IND,2009-03-01,272.985075,NaN,Influenza,9.308642,48.75555556,Cloudy,5.129032,8.254357,124.1935484,ESE,29.208444,47.2,48.75555556,0.199333,OVC,-45.155556,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
3,India,IND,2009-04-01,0.000000,NaN,Influenza,11.485507,52.67391304,Cloudy,4.977273,8.010124,119.7727273,E,29.143913,48.63043478,52.67391304,0.380870,OVC,-32.369565,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
4,India,IND,2009-05-01,0.000000,NaN,Influenza,14.056848,57.30232558,Partly Cloudy,4.852941,7.810032,246.1764706,NW,29.093953,52.20930233,57.30232558,0.451628,SCT,-25.093023,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,India,IND,2022-08-01,29703.021991,NaN,Influenza,15.858586,60.54545455,Fog,8.295455,13.350207,92.14285714,E,29.159773,59.22727273,60.54545455,0.795455,OVC,0.409091,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
164,India,IND,2022-09-01,18416.610716,NaN,Influenza,13.715278,56.6875,Fog,4.916667,7.912588,258.9189189,W,29.146250,55.95833333,56.6875,0.800417,OVC,0.375000,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
165,India,IND,2022-10-01,14643.686176,NaN,Influenza,12.395833,54.3125,Fog,3.812500,6.135609,274.5945946,W,29.221667,52.47916667,54.3125,0.795417,SCT,0.625000,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."
166,India,IND,2022-11-01,8614.339370,NaN,Influenza,10.969267,51.74468085,Fog,5.617021,9.039697,269.4871795,W,29.224043,50.23404255,51.74468085,0.612553,CLR,0.872340,"[""Fever"", ""Cough"", ""Sore throat"", ""Runny or st..."


In [174]:
monthly_cholera = pd.read_csv('Monthly/monthly_cholera.csv')
monthly_cholera

,Year,Month,Cases
0,1950,1,60
1,1950,2,57
2,1950,3,103
3,1950,4,123
4,1950,5,111
...,...,...,...
799,2020,8,3
800,2020,9,3
801,2020,10,1
802,2020,11,2


In [175]:
def distribute_cases(df):
    result = []
    for _, row in df.iterrows():
        year = row['Year']
        month = row['Month']
        total_cases = row['Cases']
        days_in_month = pd.Period(f"{year}-{month}", 'M').days_in_month
        average_cases = total_cases/days_in_month
        daily_cases = np.random.randint(0.7*average_cases, high=1.3*average_cases+1, size=days_in_month)
        daily_cases[-1] += total_cases - daily_cases.sum()
        if daily_cases[-1] < 0:
            daily_cases[-1] *= -1
        for i, cases in enumerate(daily_cases):
            result.append({
                'Year': year,
                'Month': month,
                'Day': i + 1,
                'Cases': cases
            })
    result_df = pd.DataFrame(result)
    result_df['Date'] = pd.to_datetime(result_df[['Year', 'Month', 'Day']])
    result_df.drop(columns=['Year', 'Month', 'Day'], inplace=True)
    return result_df

# Distribute cases randomly across days
cholera = distribute_cases(monthly_cholera)
cholera

,Cases,Date
0,2,1950-01-01
1,2,1950-01-02
2,1,1950-01-03
3,1,1950-01-04
4,2,1950-01-05
...,...,...
24467,0,2020-12-27
24468,0,2020-12-28
24469,0,2020-12-29
24470,0,2020-12-30


In [176]:
cholera['Code'] = 'IND'
cholera['Disease'] = 'Cholera'
cholera['Deaths'] = np.nan
cholera['Entity'] = 'India'
cholera = cholera[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
cholera

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,1950-01-01,2,NaN,Cholera
1,India,IND,1950-01-02,2,NaN,Cholera
2,India,IND,1950-01-03,1,NaN,Cholera
3,India,IND,1950-01-04,1,NaN,Cholera
4,India,IND,1950-01-05,2,NaN,Cholera
...,...,...,...,...,...,...
24467,India,IND,2020-12-27,0,NaN,Cholera
24468,India,IND,2020-12-28,0,NaN,Cholera
24469,India,IND,2020-12-29,0,NaN,Cholera
24470,India,IND,2020-12-30,0,NaN,Cholera


In [177]:
cholera = pd.merge(cholera, df, on='Date', how='inner')
cholera['Symptoms'] = '["Profuse watery diarrhea", "Nausea", "Vomiting", "Muscle cramps", "Dehydration", "Shock"]'
cholera.to_csv('Daily/daily_cholera.csv')
cholera

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2010-01-01,22,NaN,Cholera,12.678063,54.82051282,Fair,5.121212,8.241772,239.0909091,W,29.180769,47.23076923,54.82051282,0.635385,CLR,0.000000,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
1,India,IND,2010-01-02,14,NaN,Cholera,16.005291,60.80952381,Fair,3.516129,5.658647,144.6666667,CALM,29.178810,45.0952381,60.80952381,1.005000,CLR,-22.333333,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
2,India,IND,2010-01-03,22,NaN,Cholera,25.858586,78.54545455,Fair,5.666667,9.119593,185.3846154,E,29.037045,54.86363636,78.65909091,1.975227,CLR,-66.727273,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
3,India,IND,2010-01-04,20,NaN,Cholera,30.211640,86.38095238,Fair,7.413793,11.931314,252.7586207,CALM,28.847857,45.45238095,86.45238095,1.542143,CLR,-4.619048,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
4,India,IND,2010-01-05,20,NaN,Cholera,30.987654,87.77777778,Mostly Cloudy,7.545455,12.143202,149.7727273,ESE,28.945333,56.86666667,87.86666667,1.044444,BKN,-107.533333,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,India,IND,2020-12-27,0,NaN,Cholera,14.444444,58,Fog,4.958333,7.979644,168.6206897,CALM,29.101458,47.9375,58,0.750417,FEW,0.687500,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
2915,India,IND,2020-12-28,0,NaN,Cholera,13.206019,55.77083333,Haze,4.854167,7.812005,307.5609756,NW,29.180833,41.29166667,55.77083333,0.974375,CLR,0.833333,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
2916,India,IND,2020-12-29,0,NaN,Cholera,10.474537,50.85416667,Fog,5.291667,8.516091,275.952381,W,29.210417,42.04166667,50.85416667,0.935000,CLR,0.833333,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
2917,India,IND,2020-12-30,0,NaN,Cholera,9.858156,49.74468085,Fog,5.531915,8.902732,284.2857143,W,29.248936,44.23404255,49.74468085,0.685106,FEW,0.702128,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."


In [178]:
monthly_dengue = pd.read_csv('Monthly/monthly_dengue.csv')
dengue = distribute_cases(monthly_dengue)
dengue['Code'] = 'IND'
dengue['Disease'] = 'Dengue'
dengue['Deaths'] = np.nan
dengue['Entity'] = 'India'
dengue = dengue[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
dengue

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,1990-01-01,39179,NaN,Dengue
1,India,IND,1990-01-02,41641,NaN,Dengue
2,India,IND,1990-01-03,54436,NaN,Dengue
3,India,IND,1990-01-04,47675,NaN,Dengue
4,India,IND,1990-01-05,38552,NaN,Dengue
...,...,...,...,...,...,...
10952,India,IND,2019-12-27,85600,NaN,Dengue
10953,India,IND,2019-12-28,52529,NaN,Dengue
10954,India,IND,2019-12-29,78280,NaN,Dengue
10955,India,IND,2019-12-30,91176,NaN,Dengue


In [179]:
dengue = pd.merge(dengue, df, on='Date', how='inner')
dengue['Symptoms'] = '["Sudden, high fever", "Severe headaches", "Pain behind the eyes", "Severe joint and muscle pain", "Fatigue", "Nausea", "Vomiting", "Skin rash"]'
dengue.to_csv('Daily/daily_dengue.csv')
dengue

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2009-01-01,110495,NaN,Dengue,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
1,India,IND,2009-01-02,68321,NaN,Dengue,15.284553,59.51219512,Fair,6.945946,11.178389,267.5675676,WSW,29.133415,42.92682927,59.51219512,1.262683,FEW,-16.658537,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
2,India,IND,2009-01-03,98978,NaN,Dengue,20.176152,68.31707317,Fair,9.650000,15.530131,275,WNW,29.006098,45.12195122,68.34146341,1.664634,CLR,-0.658537,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
3,India,IND,2009-01-04,106943,NaN,Dengue,26.256614,79.26190476,Partly Cloudy,9.128205,14.690386,293.0769231,NW,28.940714,54.38095238,79.45238095,1.963571,SCT,-56.285714,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
4,India,IND,2009-01-05,108872,NaN,Dengue,35.656566,96.18181818,Fair,7.758621,12.486259,245.862069,WSW,28.832273,46.88636364,96.18181818,1.669318,CLR,-19.045455,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,India,IND,2019-12-27,85600,NaN,Dengue,8.629630,47.53333333,Fog,3.644444,5.865150,258.125,CALM,29.228444,43.53333333,47.53333333,0.512444,FEW,0.688889,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
4011,India,IND,2019-12-28,52529,NaN,Dengue,7.021277,44.63829787,Fog,2.085106,3.355645,261.3043478,CALM,29.246170,43.5106383,44.63829787,0.336596,CLR,0.702128,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
4012,India,IND,2019-12-29,78280,NaN,Dengue,8.703704,47.66666667,Fog,2.333333,3.755127,191.7391304,CALM,29.258958,44.85416667,47.66666667,0.314792,OVC,0.562500,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."
4013,India,IND,2019-12-30,91176,NaN,Dengue,6.544715,43.7804878,Fog,2.609756,4.199985,140,CALM,29.232195,43.58536585,43.7804878,0.164634,OVC,0.536585,"[""Sudden, high fever"", ""Severe headaches"", ""Pa..."


In [180]:
def distribute_deaths(df):
    result = []
    for _, row in df.iterrows():
        year = row['Year']
        month = row['Month']
        total_cases = row['Deaths']
        days_in_month = pd.Period(f"{int(year)}-{int(month)}", 'M').days_in_month
        average_cases = total_cases/days_in_month
        daily_cases = np.random.randint(0.7*average_cases, high=1.3*average_cases+1, size=days_in_month)
        daily_cases[-1] += total_cases - daily_cases.sum()
        if daily_cases[-1] < 0:
            daily_cases[-1] *= -1
        for i, deaths in enumerate(daily_cases):
            result.append({
                'Year': year,
                'Month': month,
                'Day': i + 1,
                'Deaths': deaths
            })
    result_df = pd.DataFrame(result)
    result_df['Date'] = pd.to_datetime(result_df[['Year', 'Month', 'Day']])
    result_df.drop(columns=['Year', 'Month', 'Day'], inplace=True)
    return result_df

In [181]:
monthly_diarrhea = pd.read_csv('Monthly/monthly_diarrhea.csv')
diarrhea = distribute_deaths(monthly_diarrhea)
diarrhea['Code'] = 'IND'
diarrhea['Disease'] = 'Diarrhea'
diarrhea['Cases'] = np.nan
diarrhea['Entity'] = 'India'
diarrhea = diarrhea[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
diarrhea

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,2000-01-01,NaN,3113,Diarrhea
1,India,IND,2000-01-02,NaN,1745,Diarrhea
2,India,IND,2000-01-03,NaN,1899,Diarrhea
3,India,IND,2000-01-04,NaN,2470,Diarrhea
4,India,IND,2000-01-05,NaN,1768,Diarrhea
...,...,...,...,...,...,...
7300,India,IND,2019-12-27,NaN,3527,Diarrhea
7301,India,IND,2019-12-28,NaN,4209,Diarrhea
7302,India,IND,2019-12-29,NaN,3427,Diarrhea
7303,India,IND,2019-12-30,NaN,3327,Diarrhea


In [182]:
diarrhea = pd.merge(diarrhea, df, on='Date', how='inner')
diarrhea['Symptoms'] = '["Frequent loose or watery stools", "Abdominal cramps", "Abdominal pain", "Fever", "Bloody stools", "Nausea", "Vomiting"]'
diarrhea.to_csv('Daily/daily_diarrhea.csv')
diarrhea

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2009-01-01,NaN,1375,Diarrhea,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000,"[""Frequent loose or watery stools"", ""Abdominal..."
1,India,IND,2009-01-02,NaN,2314,Diarrhea,15.284553,59.51219512,Fair,6.945946,11.178389,267.5675676,WSW,29.133415,42.92682927,59.51219512,1.262683,FEW,-16.658537,"[""Frequent loose or watery stools"", ""Abdominal..."
2,India,IND,2009-01-03,NaN,1928,Diarrhea,20.176152,68.31707317,Fair,9.650000,15.530131,275,WNW,29.006098,45.12195122,68.34146341,1.664634,CLR,-0.658537,"[""Frequent loose or watery stools"", ""Abdominal..."
3,India,IND,2009-01-04,NaN,1269,Diarrhea,26.256614,79.26190476,Partly Cloudy,9.128205,14.690386,293.0769231,NW,28.940714,54.38095238,79.45238095,1.963571,SCT,-56.285714,"[""Frequent loose or watery stools"", ""Abdominal..."
4,India,IND,2009-01-05,NaN,1407,Diarrhea,35.656566,96.18181818,Fair,7.758621,12.486259,245.862069,WSW,28.832273,46.88636364,96.18181818,1.669318,CLR,-19.045455,"[""Frequent loose or watery stools"", ""Abdominal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,India,IND,2019-12-27,NaN,3527,Diarrhea,8.629630,47.53333333,Fog,3.644444,5.865150,258.125,CALM,29.228444,43.53333333,47.53333333,0.512444,FEW,0.688889,"[""Frequent loose or watery stools"", ""Abdominal..."
4011,India,IND,2019-12-28,NaN,4209,Diarrhea,7.021277,44.63829787,Fog,2.085106,3.355645,261.3043478,CALM,29.246170,43.5106383,44.63829787,0.336596,CLR,0.702128,"[""Frequent loose or watery stools"", ""Abdominal..."
4012,India,IND,2019-12-29,NaN,3427,Diarrhea,8.703704,47.66666667,Fog,2.333333,3.755127,191.7391304,CALM,29.258958,44.85416667,47.66666667,0.314792,OVC,0.562500,"[""Frequent loose or watery stools"", ""Abdominal..."
4013,India,IND,2019-12-30,NaN,3327,Diarrhea,6.544715,43.7804878,Fog,2.609756,4.199985,140,CALM,29.232195,43.58536585,43.7804878,0.164634,OVC,0.536585,"[""Frequent loose or watery stools"", ""Abdominal..."


In [183]:
monthly_pneumonia = pd.read_csv('Monthly/monthly_pneumonia.csv')
pneumonia = distribute_deaths(monthly_pneumonia)
pneumonia['Code'] = 'IND'
pneumonia['Disease'] = 'Pnuemonia'
pneumonia['Cases'] = np.nan
pneumonia['Entity'] = 'India'
pneumonia = pneumonia[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
pneumonia

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,1990-01-01,NaN,1074,Pnuemonia
1,India,IND,1990-01-02,NaN,1167,Pnuemonia
2,India,IND,1990-01-03,NaN,883,Pnuemonia
3,India,IND,1990-01-04,NaN,1308,Pnuemonia
4,India,IND,1990-01-05,NaN,1517,Pnuemonia
...,...,...,...,...,...,...
10952,India,IND,2019-12-27,NaN,1524,Pnuemonia
10953,India,IND,2019-12-28,NaN,1570,Pnuemonia
10954,India,IND,2019-12-29,NaN,1455,Pnuemonia
10955,India,IND,2019-12-30,NaN,1436,Pnuemonia


In [184]:
pneumonia = pd.merge(pneumonia, df, on='Date', how='inner')
pneumonia['Symptoms'] = '["Cough", "Fever", "Chills", "Shortness of breath", "Chest pain", "Sweating", "Rapid heartbeat", "Fatigue", "Nausea", "Vomiting", "Diarrhea"]'
pneumonia.to_csv('Daily/daily_pnuemonia.csv')
pneumonia

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2009-01-01,NaN,828,Pnuemonia,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
1,India,IND,2009-01-02,NaN,739,Pnuemonia,15.284553,59.51219512,Fair,6.945946,11.178389,267.5675676,WSW,29.133415,42.92682927,59.51219512,1.262683,FEW,-16.658537,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
2,India,IND,2009-01-03,NaN,625,Pnuemonia,20.176152,68.31707317,Fair,9.650000,15.530131,275,WNW,29.006098,45.12195122,68.34146341,1.664634,CLR,-0.658537,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
3,India,IND,2009-01-04,NaN,719,Pnuemonia,26.256614,79.26190476,Partly Cloudy,9.128205,14.690386,293.0769231,NW,28.940714,54.38095238,79.45238095,1.963571,SCT,-56.285714,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
4,India,IND,2009-01-05,NaN,697,Pnuemonia,35.656566,96.18181818,Fair,7.758621,12.486259,245.862069,WSW,28.832273,46.88636364,96.18181818,1.669318,CLR,-19.045455,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,India,IND,2019-12-27,NaN,1524,Pnuemonia,8.629630,47.53333333,Fog,3.644444,5.865150,258.125,CALM,29.228444,43.53333333,47.53333333,0.512444,FEW,0.688889,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
4011,India,IND,2019-12-28,NaN,1570,Pnuemonia,7.021277,44.63829787,Fog,2.085106,3.355645,261.3043478,CALM,29.246170,43.5106383,44.63829787,0.336596,CLR,0.702128,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
4012,India,IND,2019-12-29,NaN,1455,Pnuemonia,8.703704,47.66666667,Fog,2.333333,3.755127,191.7391304,CALM,29.258958,44.85416667,47.66666667,0.314792,OVC,0.562500,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."
4013,India,IND,2019-12-30,NaN,1436,Pnuemonia,6.544715,43.7804878,Fog,2.609756,4.199985,140,CALM,29.232195,43.58536585,43.7804878,0.164634,OVC,0.536585,"[""Cough"", ""Fever"", ""Chills"", ""Shortness of bre..."


In [185]:
monthly_tuberculosis = pd.read_csv('Monthly/monthly_tuberculosis.csv')
tuberculosis = distribute_cases(monthly_tuberculosis)
tuberculosis['Code'] = 'IND'
tuberculosis['Disease'] = 'Tuberculosis'
tuberculosis['Deaths'] = np.nan
tuberculosis['Entity'] = 'India'
tuberculosis = tuberculosis[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
tuberculosis

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,2000-01-01,1,NaN,Tuberculosis
1,India,IND,2000-01-02,1,NaN,Tuberculosis
2,India,IND,2000-01-03,1,NaN,Tuberculosis
3,India,IND,2000-01-04,1,NaN,Tuberculosis
4,India,IND,2000-01-05,0,NaN,Tuberculosis
...,...,...,...,...,...,...
8396,India,IND,2022-12-27,0,NaN,Tuberculosis
8397,India,IND,2022-12-28,0,NaN,Tuberculosis
8398,India,IND,2022-12-29,0,NaN,Tuberculosis
8399,India,IND,2022-12-30,0,NaN,Tuberculosis


In [186]:
tuberculosis = pd.merge(tuberculosis, df, on='Date', how='inner')
tuberculosis['Symptoms'] = '["Cough that lasts three or more weeks", "Coughing up blood", "Chest pain or pain with breathing or coughing", "Unintentional weight loss", "Fatigue", "Fever", "Night sweats", "Chills", "Loss of appetite"]'
tuberculosis.to_csv('Daily/daily_tuberculosis.csv')
tuberculosis

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2009-01-01,0,NaN,Tuberculosis,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000,"[""Cough that lasts three or more weeks"", ""Coug..."
1,India,IND,2009-01-02,1,NaN,Tuberculosis,15.284553,59.51219512,Fair,6.945946,11.178389,267.5675676,WSW,29.133415,42.92682927,59.51219512,1.262683,FEW,-16.658537,"[""Cough that lasts three or more weeks"", ""Coug..."
2,India,IND,2009-01-03,0,NaN,Tuberculosis,20.176152,68.31707317,Fair,9.650000,15.530131,275,WNW,29.006098,45.12195122,68.34146341,1.664634,CLR,-0.658537,"[""Cough that lasts three or more weeks"", ""Coug..."
3,India,IND,2009-01-04,0,NaN,Tuberculosis,26.256614,79.26190476,Partly Cloudy,9.128205,14.690386,293.0769231,NW,28.940714,54.38095238,79.45238095,1.963571,SCT,-56.285714,"[""Cough that lasts three or more weeks"", ""Coug..."
4,India,IND,2009-01-05,1,NaN,Tuberculosis,35.656566,96.18181818,Fair,7.758621,12.486259,245.862069,WSW,28.832273,46.88636364,96.18181818,1.669318,CLR,-19.045455,"[""Cough that lasts three or more weeks"", ""Coug..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5103,India,IND,2022-12-27,0,NaN,Tuberculosis,10.173611,50.3125,Fog,5.875000,9.454873,259.1666667,W,29.228750,47.29166667,50.3125,0.467708,CLR,0.687500,"[""Cough that lasts three or more weeks"", ""Coug..."
5104,India,IND,2022-12-28,0,NaN,Tuberculosis,12.986111,55.375,Fog,5.437500,8.750786,253.0434783,W,29.211875,45.83333333,55.375,0.944792,CLR,0.812500,"[""Cough that lasts three or more weeks"", ""Coug..."
5105,India,IND,2022-12-29,0,NaN,Tuberculosis,14.791667,58.625,Fog,2.500000,4.023350,187.8125,CALM,29.234167,45.70833333,58.625,0.967292,BKN,0.458333,"[""Cough that lasts three or more weeks"", ""Coug..."
5106,India,IND,2022-12-30,0,NaN,Tuberculosis,15.591787,60.06521739,Fog,5.130435,8.256614,137.1428571,E,29.248261,51.08695652,60.06521739,0.733696,SCT,0.608696,"[""Cough that lasts three or more weeks"", ""Coug..."


In [187]:
monthly_typhoid = pd.read_csv('Monthly/monthly_typhoid.csv')
typhoid = distribute_cases(monthly_typhoid)
typhoid['Code'] = 'IND'
typhoid['Disease'] = 'Typhoid'
typhoid['Deaths'] = np.nan
typhoid['Entity'] = 'India'
typhoid = typhoid[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
typhoid

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,1990-01-01,3938,NaN,Typhoid
1,India,IND,1990-01-02,6841,NaN,Typhoid
2,India,IND,1990-01-03,5270,NaN,Typhoid
3,India,IND,1990-01-04,6154,NaN,Typhoid
4,India,IND,1990-01-05,5142,NaN,Typhoid
...,...,...,...,...,...,...
10952,India,IND,2019-12-27,2784,NaN,Typhoid
10953,India,IND,2019-12-28,2840,NaN,Typhoid
10954,India,IND,2019-12-29,3703,NaN,Typhoid
10955,India,IND,2019-12-30,3546,NaN,Typhoid


In [188]:
typhoid = pd.merge(typhoid, df, on='Date', how='inner')
typhoid['Symptoms'] = '["High fever", "Stomach pains", "Headache", "Weakness or fatigue", "Loss of appetite", "Constipation or diarrhea", "Rose-colored spots on the chest", "Cough"]'
typhoid.to_csv('Daily/daily_typhoid.csv')
typhoid

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2009-01-01,4118,NaN,Typhoid,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
1,India,IND,2009-01-02,2940,NaN,Typhoid,15.284553,59.51219512,Fair,6.945946,11.178389,267.5675676,WSW,29.133415,42.92682927,59.51219512,1.262683,FEW,-16.658537,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
2,India,IND,2009-01-03,3909,NaN,Typhoid,20.176152,68.31707317,Fair,9.650000,15.530131,275,WNW,29.006098,45.12195122,68.34146341,1.664634,CLR,-0.658537,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
3,India,IND,2009-01-04,4589,NaN,Typhoid,26.256614,79.26190476,Partly Cloudy,9.128205,14.690386,293.0769231,NW,28.940714,54.38095238,79.45238095,1.963571,SCT,-56.285714,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
4,India,IND,2009-01-05,3795,NaN,Typhoid,35.656566,96.18181818,Fair,7.758621,12.486259,245.862069,WSW,28.832273,46.88636364,96.18181818,1.669318,CLR,-19.045455,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,India,IND,2019-12-27,2784,NaN,Typhoid,8.629630,47.53333333,Fog,3.644444,5.865150,258.125,CALM,29.228444,43.53333333,47.53333333,0.512444,FEW,0.688889,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
4011,India,IND,2019-12-28,2840,NaN,Typhoid,7.021277,44.63829787,Fog,2.085106,3.355645,261.3043478,CALM,29.246170,43.5106383,44.63829787,0.336596,CLR,0.702128,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
4012,India,IND,2019-12-29,3703,NaN,Typhoid,8.703704,47.66666667,Fog,2.333333,3.755127,191.7391304,CALM,29.258958,44.85416667,47.66666667,0.314792,OVC,0.562500,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."
4013,India,IND,2019-12-30,3546,NaN,Typhoid,6.544715,43.7804878,Fog,2.609756,4.199985,140,CALM,29.232195,43.58536585,43.7804878,0.164634,OVC,0.536585,"[""High fever"", ""Stomach pains"", ""Headache"", ""W..."


In [189]:
monthly_unsafe_sanitation = pd.read_csv('Monthly/monthly_unsafe-sanitation.csv')
unsafe_sanitation = distribute_deaths(monthly_unsafe_sanitation)
unsafe_sanitation['Code'] = 'IND'
unsafe_sanitation['Disease'] = 'Unsafe Sanitation'
unsafe_sanitation['Cases'] = np.nan
unsafe_sanitation['Entity'] = 'India'
unsafe_sanitation = unsafe_sanitation[['Entity', 'Code', 'Date', 'Cases', 'Deaths', 'Disease']]
unsafe_sanitation

,Entity,Code,Date,Cases,Deaths,Disease
0,India,IND,1990-01-01,NaN,1457,Unsafe Sanitation
1,India,IND,1990-01-02,NaN,1149,Unsafe Sanitation
2,India,IND,1990-01-03,NaN,1075,Unsafe Sanitation
3,India,IND,1990-01-04,NaN,1051,Unsafe Sanitation
4,India,IND,1990-01-05,NaN,1299,Unsafe Sanitation
...,...,...,...,...,...,...
10952,India,IND,2019-12-27,NaN,1292,Unsafe Sanitation
10953,India,IND,2019-12-28,NaN,1511,Unsafe Sanitation
10954,India,IND,2019-12-29,NaN,941,Unsafe Sanitation
10955,India,IND,2019-12-30,NaN,1071,Unsafe Sanitation


In [190]:
unsafe_sanitation = pd.merge(unsafe_sanitation, df, on='Date', how='inner')
unsafe_sanitation['Symptoms'] = '["Contaminated water", "Poor hygiene practices", "Open defecation", "Lack of access to clean water", "Inadequate sewage systems"]'
unsafe_sanitation.to_csv('Daily/daily_unsafe_sanitation.csv')
unsafe_sanitation

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2009-01-01,NaN,574,Unsafe Sanitation,10.707071,51.27272727,Fair,6.093750,9.806916,279.0625,W,29.178864,48.13636364,51.27272727,0.306136,CLR,-31.500000,"[""Contaminated water"", ""Poor hygiene practices..."
1,India,IND,2009-01-02,NaN,559,Unsafe Sanitation,15.284553,59.51219512,Fair,6.945946,11.178389,267.5675676,WSW,29.133415,42.92682927,59.51219512,1.262683,FEW,-16.658537,"[""Contaminated water"", ""Poor hygiene practices..."
2,India,IND,2009-01-03,NaN,524,Unsafe Sanitation,20.176152,68.31707317,Fair,9.650000,15.530131,275,WNW,29.006098,45.12195122,68.34146341,1.664634,CLR,-0.658537,"[""Contaminated water"", ""Poor hygiene practices..."
3,India,IND,2009-01-04,NaN,912,Unsafe Sanitation,26.256614,79.26190476,Partly Cloudy,9.128205,14.690386,293.0769231,NW,28.940714,54.38095238,79.45238095,1.963571,SCT,-56.285714,"[""Contaminated water"", ""Poor hygiene practices..."
4,India,IND,2009-01-05,NaN,742,Unsafe Sanitation,35.656566,96.18181818,Fair,7.758621,12.486259,245.862069,WSW,28.832273,46.88636364,96.18181818,1.669318,CLR,-19.045455,"[""Contaminated water"", ""Poor hygiene practices..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,India,IND,2019-12-27,NaN,1292,Unsafe Sanitation,8.629630,47.53333333,Fog,3.644444,5.865150,258.125,CALM,29.228444,43.53333333,47.53333333,0.512444,FEW,0.688889,"[""Contaminated water"", ""Poor hygiene practices..."
4011,India,IND,2019-12-28,NaN,1511,Unsafe Sanitation,7.021277,44.63829787,Fog,2.085106,3.355645,261.3043478,CALM,29.246170,43.5106383,44.63829787,0.336596,CLR,0.702128,"[""Contaminated water"", ""Poor hygiene practices..."
4012,India,IND,2019-12-29,NaN,941,Unsafe Sanitation,8.703704,47.66666667,Fog,2.333333,3.755127,191.7391304,CALM,29.258958,44.85416667,47.66666667,0.314792,OVC,0.562500,"[""Contaminated water"", ""Poor hygiene practices..."
4013,India,IND,2019-12-30,NaN,1071,Unsafe Sanitation,6.544715,43.7804878,Fog,2.609756,4.199985,140,CALM,29.232195,43.58536585,43.7804878,0.164634,OVC,0.536585,"[""Contaminated water"", ""Poor hygiene practices..."


In [191]:
dfs = [cholera, daily_covid, dengue, diarrhea, pneumonia, tuberculosis, typhoid, unsafe_sanitation]
concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)
concatenated_df

,Entity,Code,Date,Cases,Deaths,Disease,Average Temperature (C),Average Temperature (F),Most Repeated Weather Phrase,Average Wind Speed (mph),Average Wind Speed (kph),Average Wind Degree,Most Repeated Wind Direction,Average Pressure,Average Dew Point,Average Heat Index,Average Visibility,Most Repeated Cloud Cover,Average UV Index,Symptoms
0,India,IND,2010-01-01,22.0,NaN,Cholera,12.678063,54.82051282,Fair,5.121212,8.241772,239.0909091,W,29.180769,47.23076923,54.82051282,0.635385,CLR,0.000000,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
1,India,IND,2010-01-02,14.0,NaN,Cholera,16.005291,60.80952381,Fair,3.516129,5.658647,144.6666667,CALM,29.178810,45.0952381,60.80952381,1.005000,CLR,-22.333333,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
2,India,IND,2010-01-03,22.0,NaN,Cholera,25.858586,78.54545455,Fair,5.666667,9.119593,185.3846154,E,29.037045,54.86363636,78.65909091,1.975227,CLR,-66.727273,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
3,India,IND,2010-01-04,20.0,NaN,Cholera,30.211640,86.38095238,Fair,7.413793,11.931314,252.7586207,CALM,28.847857,45.45238095,86.45238095,1.542143,CLR,-4.619048,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
4,India,IND,2010-01-05,20.0,NaN,Cholera,30.987654,87.77777778,Mostly Cloudy,7.545455,12.143202,149.7727273,ESE,28.945333,56.86666667,87.86666667,1.044444,BKN,-107.533333,"[""Profuse watery diarrhea"", ""Nausea"", ""Vomitin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29173,India,IND,2019-12-27,NaN,1292.0,Unsafe Sanitation,8.629630,47.53333333,Fog,3.644444,5.865150,258.125,CALM,29.228444,43.53333333,47.53333333,0.512444,FEW,0.688889,"[""Contaminated water"", ""Poor hygiene practices..."
29174,India,IND,2019-12-28,NaN,1511.0,Unsafe Sanitation,7.021277,44.63829787,Fog,2.085106,3.355645,261.3043478,CALM,29.246170,43.5106383,44.63829787,0.336596,CLR,0.702128,"[""Contaminated water"", ""Poor hygiene practices..."
29175,India,IND,2019-12-29,NaN,941.0,Unsafe Sanitation,8.703704,47.66666667,Fog,2.333333,3.755127,191.7391304,CALM,29.258958,44.85416667,47.66666667,0.314792,OVC,0.562500,"[""Contaminated water"", ""Poor hygiene practices..."
29176,India,IND,2019-12-30,NaN,1071.0,Unsafe Sanitation,6.544715,43.7804878,Fog,2.609756,4.199985,140,CALM,29.232195,43.58536585,43.7804878,0.164634,OVC,0.536585,"[""Contaminated water"", ""Poor hygiene practices..."


In [192]:
concatenated_df.to_csv('Daily/daily_final.csv')